
#Proyek Analisis Data : E-Commerce Public Dataset


*   Nama        : Akbar Wahyu Pradana
*   Email       : m200b4ky0281@bangkit.academy
*   ID Dicoding : Akbar Wahyu Pradana M200B4KY0281









## Menetukaan Pertanyaan Bisnis



1. 5 State (kota) mana yang memiliki order (pesanan) terbanyak dan kontribusinya
2. 10 terbanyak order (pesanan) berdasarkan kategori pesananan




## Import Semua Packages/Library yang Digunakan

In [ ]:
!pip install pandas
!pip install plotly
!pip install streamlit -q
!pip install pipreqs

import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from google.colab import drive





In [ ]:
drive.mount('/content/gdrive')


Mounted at /content/gdrive


## Data Wrangling

### Gathering Data

In [ ]:
@st.cache_resource
def load_customers_data():
    return pd.read_csv("/content/gdrive/MyDrive/E-Commerce Public Dataset/customers_dataset.csv", sep=',')

@st.cache_resource
def load_products_data():
    return pd.read_csv("/content/gdrive/MyDrive/E-Commerce Public Dataset/products_dataset.csv", sep=',')

@st.cache_resource
def load_orders_data():
    return pd.read_csv("/content/gdrive/MyDrive/E-Commerce Public Dataset/order_items_dataset.csv", sep=',')

In [ ]:
customers_data = load_customers_data()
products_data = load_products_data()
orders_data = load_orders_data()

2024-10-16 14:18:25.098 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 14:18:25.328 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-10-16 14:18:25.329 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 14:18:25.332 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 14:18:25.838 Thread 'Thread-11': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 14:18:25.841 Thread 'Thread-11': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 14:18:27.127 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 14:18:27.132 Thread 'MainThread': miss

###Assesing Data

#### Memeriksa Missing Values

In [ ]:
#CUSTOMER
print("Missing Values pada Customers Dataset")
missing_values_customer = customers_data.isnull().sum()

print(missing_values_customer)

Missing Values pada Customers Dataset
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64


In [ ]:
#PRODUCTS
print("Missing Values pada Product Dataset")
missing_values_product = products_data.isnull().sum()

print(missing_values_product)

Missing Values pada Product Dataset
product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64


In [ ]:
#Order
print("Missing Values pada Order Dataset")
missing_values_order = orders_data.isnull().sum()

print(missing_values_order)

Missing Values pada Order Dataset
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64


Disini yang memiliki Missing Values pada product Dataset. kita dapat mengisinya dengan value lainnya, memprediksi atau menghapusnya. Disini saya akan menghapus baris yang memiliki values yang kosong

#### Memeriksa Duplicated Values

In [ ]:
#Customewr
print("Data Duplikat pada Customer Dataset")
duplikat_customer = customers_data.duplicated().sum()

print(duplikat_customer)

Data Duplikat pada Customer Dataset
0


In [ ]:
#Order
print("Data Duplikat pada Order Dataset")
duplikat_order = orders_data.duplicated().sum()

print(duplikat_order)

Data Duplikat pada Order Dataset
0


In [ ]:
#Product
print("Data Duplikat pada Product Dataset")
duplikat_product = products_data.duplicated().sum()

print(duplikat_product)

Data Duplikat pada Product Dataset
0


###Cleaning Data

In [ ]:
#Disini saya hanya cleaning dataset products karena hanya dataset ini yang memiliki missing values
products_data = products_data.dropna()

## Exploratory Data Analysis

In [ ]:
#Judul Dashboard
st.title("Business Orders Dashboard")

2024-10-16 14:18:29.903 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 14:18:29.906 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [ ]:
# Mengelompokkan kota (state) dan menghitung jumlah pesanan
state_order_counts = customers_data.groupby("customer_state")["customer_id"].count().reset_index()
state_order_counts.columns = ["State", "Number of Orders"]
state_order_counts = state_order_counts.sort_values(by="Number of Orders", ascending=False)
# Menghitung jumlah total pesanan untuk setiap kota
total_orders = state_order_counts['Number of Orders'].sum()

# Menghitung persentase dan menggabungkan yang kurang dari 2% menjadi "Other"
state_order_counts['Percentage'] = (state_order_counts['Number of Orders'] / total_orders) * 100
state_order_counts['State'] = state_order_counts['State'].where(state_order_counts['Percentage'] >= 1.5, 'Other')

# Mengelompokkan dan menjumlahkan untuk kategori "Other"
state_order_counts_grouped = state_order_counts.groupby('State')['Number of Orders'].sum().reset_index()

# Membuat pie chart untuk semua kota dengan kategori "Other"
fig_pie_chart = px.pie(state_order_counts_grouped, values="Number of Orders", names="State",
                       title="States Contribution by Number of Orders (Including 'Other'[sum thath below 1.5%])",
                       hole=0.4, labels={'State':'State', 'Number of Orders':'Number of Orders'},
                       hover_data=["Number of Orders"])


2024-10-16 14:18:29.927 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 14:18:29.930 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
# 2. Berapa total harga pesanan per kategori produk?
# Menggabungkan orders_data dan products_data
merged_data = pd.merge(orders_data, products_data, on="product_id", how="left")

# Mengelompokkan berdasarkan kategori produk dan menghitung total harga pesanan
category_price_totals = merged_data.groupby("product_category_name")["price"].sum().reset_index()
category_price_totals.columns = ["Product Category", "Total Order Price"]
category_price_totals = category_price_totals.sort_values(by="Total Order Price", ascending=False)

# Menghitung jumlah total harga pesanan untuk setiap kategori produk
total_price = category_price_totals['Total Order Price'].sum()

# Menghitung persentase dan menggabungkan yang kurang dari 2% menjadi "Other"
category_price_totals['Percentage'] = (category_price_totals['Total Order Price'] / total_price) * 100
category_price_totals['Product Category'] = category_price_totals['Product Category'].where(category_price_totals['Percentage'] >= 1.5, 'Other')

# Mengelompokkan dan menjumlahkan untuk kategori "Other"
category_price_totals_grouped = category_price_totals.groupby('Product Category')['Total Order Price'].sum().reset_index()

# Membuat pie chart untuk semua kategori produk dengan kategori "Other"
fig_pie_chart_category = px.pie(category_price_totals_grouped, values="Total Order Price", names="Product Category",
                                title="Product Categories Contribution by Total Order Price (Including 'Other'[sum that below 1.5%])",
                                hole=0.4, labels={'Product Category':'Product Category', 'Total Order Price':'Total Order Price'},
                                hover_data=["Total Order Price"])


2024-10-16 14:18:29.968 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 14:18:29.971 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


##Visualization & Explanatory Analysis

###Pertanyaan 1

In [ ]:
# Menampilkan subheader
st.header("1. 5 kota (state) yang memiliki pesanan (orders) terbanyak")

# Menampilkan 5 kota (state) yang memiliki pesanan terbanyak
st.subheader("Top 5 States by Number of Orders")
st.write(state_order_counts.head(5))

# Menampilkan diagram pie dengan kontribusi persentase
st.subheader("Pie Chart: Top 5 States by Number of Orders")


# Menampilkan pie chart di Streamlit
st.plotly_chart(fig_pie_chart)


2024-10-16 14:18:30.099 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 14:18:30.102 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 14:18:30.157 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 14:18:30.160 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 14:18:30.847 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 14:18:30.850 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 14:18:30.852 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 14:18:30.856 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

### Pertanyaan 2

In [ ]:

# Menampilkan subheader
st.header("2. Berapa total harga pesanan per kategori produk?")

# Menampilkan 10 kategori teratas berdasarkan total harga pesanan
st.subheader("Top 10 Product Categories by Total Order Price")
st.write(category_price_totals.head(10))


# Membuat bar chart untuk total harga berdasarkan kategori produk
fig_category_price = px.bar(category_price_totals.head(10), x="Product Category", y="Total Order Price",
                            title="Top 10 Product Categories by Total Order Price",
                            color="Total Order Price")

st.plotly_chart(fig_category_price)

# Menampilkan diagram pie dengan persentase kontribusi
st.subheader("Pie Chart: Top 10 Product Categories by Total Order Price")


# Menampilkan pie chart di Streamlit
st.plotly_chart(fig_pie_chart_category)



2024-10-16 14:18:30.876 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 14:18:30.878 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 14:18:30.882 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 14:18:30.884 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 14:18:30.942 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 14:18:30.945 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 14:18:30.947 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 14:18:30.949 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

##Conclusion

1. 5 kota (state) yang memiliki pesanan (orders) terbanyak secara berturut-urut adalah  

<table>
    <tr>
        <th>Kode State</th>
        <th>Total</th>
        <th>Persen</th>
    </tr>
    <tr>
        <td>SP</td>
        <td>41,746</td>
        <td>42% </td>
    </tr>
    <tr>
        <td>RJ</td>
        <td>12,852</td>
        <td>12.9% </td>
    </tr>
    <tr>
        <td>MG</td>
        <td>11,635</td>
        <td>11.7% </td>
    </tr>
    <tr>
        <td>RS</td>
        <td>5,466</td>
        <td> 5.5%</td>
    </tr>
    <tr>
        <td>PR</td>
        <td>5,045</td>
        <td>5.07% </td>
    </tr>
</table>


2. Berapa total harga pesanan per kategori produk?  


<table>
    <tr>
        <th>Kategori</th>
        <th>Total Harga</th>
        <td>Persen </td>
    </tr>
    <tr>
        <td>beleza_saude</td>
        <td>1,258,681.34</td>
        <td>9.38% </td>
    </tr>
    <tr>
        <td>relogios_presentes</td>
        <td>1,205,005.68</td>
        <td>8.98% </td>
    </tr>
    <tr>
        <td>cama_mesa_banho</td>
        <td>1,036,988.68</td>
        <td>7.73% </td>
    </tr>
    <tr>
        <td>esporte_lazer</td>
        <td>988,048.97</td>
        <td>7.37% </td>
    </tr>
    <tr>
        <td>informatica_acessorios</td>
        <td>911,954.32</td>
        <td>6.8% </td>
    </tr>
    <tr>
        <td>moveis_decoracao</td>
        <td>729,762.49</td>
        <td>5.44% </td>
    </tr>
    <tr>
        <td>cool_stuff</td>
        <td>635,290.85</td>
        <td>4.74% </td>
    </tr>
    <tr>
        <td>utilidades_domesticas</td>
        <td>632,248.66</td>
        <td>4.71% </td>
    </tr>
    <tr>
        <td>automotivo</td>
        <td>592,720.11</td>
        <td>4.42% </td>
    </tr>
    <tr>
        <td>ferramentas_jardim</td>
        <td>485,256.46</td>
        <td>3.62% </td>
    </tr>
</table>